In [ ]:
from caveclient import CAVEclient
import pandas as pd
import numpy as np 
import os
import cloudvolume
import utils

In [ ]:
client = CAVEclient()
dataset = 'fanc_production_mar2021'
client = CAVEclient(dataset)

In [ ]:
pmns = pd.read_csv('~/Code/connectomics/connectomics/annotations_pMNs/pmn_ann_table.csv')

In [ ]:
print(pmns.shape)
pmns.head()

In [ ]:
pt_ints = []
for i in pmns['pt_position'].tolist():
    #print(i)
    i = i.split('[')[1]
    i = i.split(']')[0].split(',')
    i = [j for j in i if j != '']
    
    pt_ints.append(list(map(int, i)))

print(pt_ints[0])
type(pt_ints[0])
pmns['pt_position'] = pt_ints
pmns.head()

In [ ]:
pt_arr = np.array(pt_ints)
# pt_midx = pd.MultiIndex.from_arrays([pt_arr[:,0],pt_arr[:,1],pt_arr[:,2]],names=['x','y','z'])
pmns.index = pd.MultiIndex.from_arrays([pt_arr[:,0],pt_arr[:,1],pt_arr[:,2]],names=['x','y','z'])

In [ ]:
pmns['classification_system'] = pmns.classification_system.fillna('unidentified')
pmns['cell_type'] = pmns.cell_type.fillna('unidentified')
pmns_ann_table = pmns.drop(columns=['Unnamed: 0'])
pmns_ann_table.head()


# For the purposes of checking, get segIDs for points and create a csv file

In [ ]:
# Find segIDs from points
cv = cloudvolume.CloudVolume(client.info.get_datastack_info()['segmentation_source'], use_https=True)
segIDs = utils.segIDs_from_pts_service(pmns_ann_table.pt_position.values,cv)


In [ ]:
df3.loc[df3.index.duplicated()]
df4 = df3[~df3.index.duplicated(keep='last')]

# Get rid of duplicates

In [ ]:
pmns_ann_table['pt_root_id'] = segIDs
utils.save_df_csv(pmns_ann_table,'premotor_annotation_table')

df4 = pmns_ann_table[~pmns_ann_table.index.duplicated(keep='last')]
utils.save_df_csv(df4,'premotor_annotation_table_nodups')

In [ ]:
df4.classification_system.value_counts()

In [ ]:
df5 = df4[['pt_position','classification_system','cell_type']]
df5.iloc[280:284,:]

In [ ]:
description = 'Primary neurite points T1 premotor neurons. The classification_system is a place holder that indicates neurites that run through common tracts. With comprehensive confidence in hemilineages, the classification_system can used to desigate developmental origin. Cell_type currently indicates hemilineage, with _ indicating uncertainty, but can be used in the future to rank neurons with in a class. v2 updates: annotation points can be in 4x4x45nm voxels, if dropped in a ng instance from the dash app, as in v1. This leads to incorrect segids. This is corrected in v2. Maintained by Tony Azevedo.'
client.annotation.create_table(table_name='t1l_local_premotor_table_v2',
                            schema_name='cell_type_local',
                            voxel_resolution=[4.3,4.3,45],
                            description = description)

In [ ]:
client.annotation.post_annotation_df(table_name='t1l_local_premotor_table_v2',
                                     df=df5,
                                     position_columns=["pt_position"]) 

### Run this the next day after you upload the table

In [ ]:
premotor_neuron_table = client.materialize.query_table('t1l_local_premotor_table_v0')

In [ ]:
motor_neuron_table.head()

In [ ]:
client.materialize.get_tables()

# Update

In [ ]:
# Check if all of the pt_positions are the same between the tables
unmatched_pt_position = []
t1_pt_position_tuplelist = t1_mns_df['pt_position'].to_list()
t1_pt_position_tuplelist = [tuple(item) for item in t1_mns_df['pt_position'].to_list()]
    
idx = []
for pt_position in mns_ann_table['pt_position'].to_list():
    if tuple(pt_position) not in t1_pt_position_tuplelist:
        unmatched_pt_position = unmatched_pt_position + [tuple(pt_position)]
    else:
        idx = idx +[t1_mns_df.loc[t1_pt_position_tuplelist.index(tuple(pt_position)),'id']]
print(unmatched_pt_position)
mns_ann_table['id'] = idx
mns_ann_table = mns_ann_table[['id','pt_position','classification_system','cell_type']]
mns_ann_table